## 라이브러리 import

In [ ]:
import pandas            as pd
import numpy             as np
import seaborn           as sns
import scipy.stats       as sp
import matplotlib.pyplot as plt

.

.

.



## 데이터 불러오기

In [ ]:
FeatureData = pd.read_csv('./ProcessedData/FeatureData.csv', sep=',', header=None)
FeatureData.shape

특징데이터 정상/고장 분리

In [ ]:
NoOfData = int(FeatureData.shape[0]/2)
Normal_FeatureData   = FeatureData.iloc[:NoOfData, :]
Abnormal_FeatureData = FeatureData.iloc[NoOfData:, :]

print(Normal_FeatureData.shape)
print(Abnormal_FeatureData.shape)

.

.

.



## P-value 계산

In [ ]:
NoOfFeature = FeatureData.shape[1] # 추출된 Feature 갯수

P_value = np.zeros((NoOfFeature , 2))

# 특징값 각각 T-검정 수행
for i in np.arange(NoOfFeature):
    
    T_test       = np.array(sp.ttest_ind(Normal_FeatureData.iloc[:, i] , Abnormal_FeatureData.iloc[:, i]))
    P_value[i,0] = i          # Feature Index
    P_value[i,1] = T_test[1]  # P값 (P-value)
    
P_value      = pd.DataFrame(P_value)
P_value

In [ ]:
P_value_Rank = P_value.sort_values([1],ascending=True)  # P-value 기준 오름차순 정렬

P_value_Rank

In [ ]:
# 특징값 저장
path1 = './ProcessedData/P_value.csv'        # path = '파일 경로/저장할 파일 이름'
path2 = './ProcessedData/P_value_Rank.csv'   # path = '파일 경로/저장할 파일 이름'
P_value.to_csv(path1, sep=',', header=None, index=None)
P_value_Rank.to_csv(path2, sep=',', header=None, index=None)

.

.

.



## PDF(Probabilistic Density Function) 그래프 확인 (정상 vs 고장)

In [ ]:
# P-value 낮은 순위
FeatureRank = 1

# PDF 그래프
sns.distplot(Normal_FeatureData.iloc  [:, int(P_value_Rank.iloc[FeatureRank-1,0])], label = 'Normal')
sns.distplot(Abnormal_FeatureData.iloc[:, int(P_value_Rank.iloc[FeatureRank-1,0])], label = 'Abnormal')
plt.legend(loc='upper right')
plt.show()

## P-value 기반 구분성(유의수준↓) 상위 특징값 선택

In [ ]:
Rank = 30

Normal   = np.zeros((NoOfData, Rank))
Abnormal = np.zeros((NoOfData, Rank))

for i in range(Rank):
    
    index          = int(P_value_Rank.iloc[i,0])
    Normal  [:, i] = Normal_FeatureData.iloc  [:, index].values
    Abnormal[:, i] = Abnormal_FeatureData.iloc[:, index].values

# 정상, 고장 특징값 합치기    
FeatureSelected = pd.DataFrame(np.concatenate([Normal, Abnormal] , axis=0))

print("Selected Feature Data Size :", FeatureSelected.shape)
print("= 데이터 %d개(정상/고장 각 %d개씩)가 각각 %d개의 최종 선택된 특징값으로 구성됨"
      %(FeatureSelected.shape[0], FeatureSelected.shape[0]/2, FeatureSelected.shape[1]))

최종 선택된 특징 데이터 저장 (.csv 파일)

In [ ]:
path = './ProcessedData/FeatureSelected.csv'   # path = '파일 경로//저장할 파일 이름'
FeatureSelected.to_csv(path, sep=',', header=None, index=None)